In [19]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import numpy as np
import torch.utils.data as td
import random,time
import matplotlib.pyplot as plt
num_epochs = 30



In [20]:
def cifar_loaders(batch_size, shuffle_test=False): 

    train = datasets.CIFAR100('./', train=True, download=True, 
        transform=transforms.Compose([
                                      transforms.Resize(32),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                    ]))
    test = datasets.CIFAR100('./', train=False, 
        transform=transforms.Compose([
                                      transforms.Resize(32),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                    ]))
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size,
        shuffle=True, pin_memory=True)
    test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size,
        shuffle=shuffle_test, pin_memory=True)
    return train_loader, test_loader

batch_size = 128
test_batch_size = 64
input_size = 3072




In [21]:
train_loader, _ = cifar_loaders(batch_size)
_, test_loader = cifar_loaders(test_batch_size)

import torch.nn as nn
import torch.nn.functional as F

class SevenLayerFC_Net(nn.Module):
    def __init__(self, D_in,H,D_out):
        """
        In the constructor we instantiate three nn.Linear modules and assign them as
        member variables.
        """
        super(SevenLayerFC_Net, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, 2048)
        self.linear2 = torch.nn.Linear(2048, 1024)
        self.linear3 = torch.nn.Linear(1024, 512)
        self.linear4 = torch.nn.Linear(512, 256)
        self.linear5 = torch.nn.Linear(256, 128)
        self.linear6 = torch.nn.Linear(128, D_out)
        #self.linear7 = torch.nn.Linear(256, D_out)
        # self.drop1 = torch.nn.Dropout(p=0.2)

        
    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        x = F.relu(self.linear4(x))
        x = F.relu(self.linear5(x))
        #x = F.relu(self.linear6(x))
        x = self.linear6(x)
        return F.log_softmax(x)  
      


Files already downloaded and verified
Files already downloaded and verified


In [22]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = batch_size, input_size, 300, 100
num_epochs = 100

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Construct our model by instantiating the class defined above
model = SevenLayerFC_Net(D_in, H, D_out)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

#print(model)



SevenLayerFC_Net(
  (linear1): Linear(in_features=3072, out_features=2048, bias=True)
  (linear2): Linear(in_features=2048, out_features=1024, bias=True)
  (linear3): Linear(in_features=1024, out_features=512, bias=True)
  (linear4): Linear(in_features=512, out_features=256, bias=True)
  (linear5): Linear(in_features=256, out_features=128, bias=True)
  (linear6): Linear(in_features=128, out_features=100, bias=True)
)

In [23]:
# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
criterion = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')


In [24]:
time1 = time.time()
for epoch in range(num_epochs):
    avg_loss_epoch = 0
    batch_loss = 0
    total_batches = 0

    for i, (images, labels) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)

        images = images.reshape(-1, 32*32*3).to(device)            
        labels = labels.to(device)

        #print(images.shape)
        outputs = model(images)

        loss = criterion(outputs, labels)    
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()   
        #scheduler.step(loss)
        total_batches += 1     
        batch_loss += loss.item()

    avg_loss_epoch = batch_loss/total_batches
    print ('Epoch [{}/{}], Average Loss:for epoch[{}, {:.4f}]' 
                   .format(epoch+1, num_epochs, epoch+1, avg_loss_epoch ))
    correct = 0.
    total = 0.
    for images, labels in train_loader:
        images = images.reshape(-1, 3*32*32).to(device)
        labels = labels.to(device)
        #print(labels)
        outputs_test = model(images)
        _, predicted = torch.max(outputs_test.data, 1)
        #print(predicted)
        total += labels.size(0) 
        
        correct += (predicted == labels).sum().item()
        
    print('Training Accuracy of the network on the 50000 training images: %d %%' % (     100 * correct / total))
time2 = time.time()


print("Training time in seconds: ", time2 - time1)

# Test the Model
correct = 0.
total = 0.
for images, labels in test_loader:
    images = images.reshape(-1, 3*32*32).to(device)
    labels = labels.to(device)
    outputs_test = model(images)
    _, predicted = torch.max(outputs_test.data, 1)
    #print(predicted)
    total += labels.size(0) 
    
    correct += (predicted == labels).sum().item()
    
print('Accuracy of the network on the 10000 test images: %d %%' % (     100 * correct / total))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch [1/100], Average Loss:for epoch[1, 3.9848]
Training Accuracy of the network on the 50000 training images: 12 %
Epoch [2/100], Average Loss:for epoch[2, 3.5980]
Training Accuracy of the network on the 50000 training images: 17 %
Epoch [3/100], Average Loss:for epoch[3, 3.3900]
Training Accuracy of the network on the 50000 training images: 21 %
Epoch [4/100], Average Loss:for epoch[4, 3.2312]
Training Accuracy of the network on the 50000 training images: 24 %
Epoch [5/100], Average Loss:for epoch[5, 3.0907]
Training Accuracy of the network on the 50000 training images: 27 %
Epoch [6/100], Average Loss:for epoch[6, 2.9556]
Training Accuracy of the network on the 50000 training images: 30 %
Epoch [7/100], Average Loss:for epoch[7, 2.8072]
Training Accuracy of the network on the 50000 training images: 31 %
Epoch [8/100], Average Loss:for epoch[8, 2.6843]
Training Accuracy of the network on the 50000 training images: 35 %
Epoch [9/100], Average Loss:for epoch[9, 2.5517]
Training Accura